In [ ]:
# Environment & determinism (document demo)
import os, sys, json, platform, random, datetime
try:
    import numpy as np
except Exception:
    np = None
try:
    import torch
except Exception:
    torch = None
IN_COLAB = 'google.colab' in sys.modules
SEED = 123
random.seed(SEED)
if np is not None:
    np.random.seed(SEED)
if torch is not None:
    torch.manual_seed(SEED)
    try:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass
print(json.dumps({
    'in_colab': IN_COLAB,
    'python': sys.version.split()[0],
    'platform': platform.platform(),
    'timestamp': datetime.datetime.utcnow().isoformat()+'Z'
}, indent=2))

# Restoria Document Demo

Explore document restoration and enhancement using Restoria backends.

## Optional extras (Colab only)

If you're on Google Colab and want optional acceleration or metrics:
- ORT (ONNX Runtime) for CPU speedups
- Metrics: ArcFace (identity), LPIPS/DISTS (perceptual)

Run the next cell to install. Safe to skip; notebook works without it.

In [ ]:
import os, sys, subprocess, shlex

# Detect Colab
IN_COLAB = "COLAB_GPU" in os.environ or "google.colab" in sys.modules

if IN_COLAB and not os.environ.get("NB_CI_SMOKE"):
    # Keep quiet and resilient; skip if already present
    def pip_install(pkgs):
        cmd = [sys.executable, "-m", "pip", "install", "-q"] + pkgs
        try:
            subprocess.check_call(cmd)
        except Exception as e:
            print(f"[WARN] pip install failed for {pkgs}: {e}")

    extras = []
    # Optional ORT CPU
    extras += ["onnxruntime>=1.17,<2"]
    # Optional metrics
    extras += [
        "lpips>=0.1.4",
        "git+https://github.com/richzhang/PerceptualSimilarity.git@master#egg=lpips-extra",
        "torchdists>=0.1.0; python_version>='3.9'",
    ]
    # ArcFace deps are heavy; skip by default. Uncomment to enable:
    # extras += ["insightface>=0.7.3", "onnxruntime-gpu; platform_system=='Linux'"]

    print("[INFO] Installing optional extras (Colab)…")
    pip_install(extras)
else:
    print("[INFO] Extras install skipped (not in Colab or CI smoke mode).")

# Document Preset Demo (Dry-Run)

This notebook shows how to invoke the GFPP CLI with the `document` preset in dry-run mode.

Notes:
- Uses `assets/gfpgan_logo.png` for a quick, local run.
- `--dry-run` avoids downloading any model weights.
- Remove dry-run to perform actual restoration (see docs for required extras).

In [ ]:
from src.gfpp.cli import cmd_run
import os

inp = os.path.join('assets','gfpgan_logo.png')
out = os.path.join('results','dry_demo','notebooks','document')
os.makedirs(out, exist_ok=True)

args = [
    '--input', inp,
    '--output', out,
    '--preset', 'document',
    '--metrics', 'off',
    '--dry-run'
]

code = cmd_run(args)
print('exit', code)
print('Planned results directory:', out)

In [ ]:
# Optional: print any manifest info if present
from glob import glob
import json

manifests = glob(os.path.join(out, 'manifest*.json'))
print('Found manifests:', manifests)
if manifests:
    try:
        with open(manifests[0], 'r') as f:
            data = json.load(f)
        env = data.get('env', {})
        print('runtime:', env.get('runtime'))
        print('images recorded:', len(data.get('images', [])))
    except Exception as e:
        print('Manifest read skipped:', e)

In [ ]:
# Quickstart (dry-run) for document demo
import os, sys
from pathlib import Path
if 'restoria' not in sys.modules:
    sys.path.insert(0, str(Path.cwd() / 'src'))
from restoria.cli.main import main as restoria_main

# Use repo asset or blank
img = 'assets/gfpgan_logo.png' if os.path.exists('assets/gfpgan_logo.png') else 'tmp/blank.png'
os.makedirs('results/dry_demo/document', exist_ok=True)
out_dir = 'results/dry_demo/document'
rc = restoria_main(['run','--input', img,'--output', out_dir,'--dry-run','--device','auto'])
print('rc:', rc)

In [ ]:
# Optional: print any manifest info if present
import json, os
mp = os.path.join('results/dry_demo/document','manifest.json')
if os.path.exists(mp):
    m = json.load(open(mp))
    print('device:', m.get('device'))
    print('args.device:', m.get('args',{}).get('device'))

### Real run (optional)

Switch off dry-run to restore with document-friendly parameters. We recommend starting with `--metrics fast` to get identity scores:

```python
from restoria.cli.main import main as restoria_main
img = 'assets/gfpgan_logo.png'
out_dir = 'results/real_demo/document'
restoria_main(['run','--input', img,'--output', out_dir,'--device','auto','--metrics','fast'])
```

## Plan and outputs summary

After a run (dry-run or normal), the output folder contains:
- manifest.json – arguments and runtime info
- metrics.json – per-image records and optional metrics

Run the next cell to print a short summary.

In [ ]:
import json, os, glob

def summarize_out(out_dir):
    man_p = os.path.join(out_dir, 'manifest.json')
    met_p = os.path.join(out_dir, 'metrics.json')
    print(f"Out: {out_dir}")
    if os.path.exists(man_p):
        man = json.load(open(man_p))
        plan = man.get('plan') or {}
        env = man.get('env') or {}
        print("- device:", (env.get('resolved_device') or env.get('device') or 'unknown'))
        if plan:
            print("- plan.backend:", plan.get('backend'))
            print("- plan.reason:", plan.get('reason'))
    else:
        print("- manifest.json not found")
    if os.path.exists(met_p):
        data = json.load(open(met_p))
        rows = data.get('records') or data
        n = len(rows) if isinstance(rows, list) else 0
        print(f"- metrics records: {n}")
        if n:
            m = rows[0].get('metrics') or {}
            for k in ['arcface_cosine','lpips','dists']:
                if k in m:
                    print(f"  * {k} = {m[k]}")
    else:
        print("- metrics.json not found")

# Example: set this to the same out_dir used in the dry-run cell above
out_dir = globals().get('out_dir', 'results/dry_demo/doc_demo')
try:
    summarize_out(out_dir)
except Exception as e:
    print("[WARN] summary failed:", e)